# Extracting Probabilities

In this notebook we run repetition codes and extract the exact probabilities rather than sampling.

In [1]:
from copy import deepcopy

from qiskit.providers.aer import AerSimulator
from topological_codes import RepetitionCode

from qiskit import QuantumCircuit, transpile

First, let's get a code circuit. This will have multiple rounds, and hence qubits with repeated measurement.

In [2]:
code = RepetitionCode(3,2)
qc = code.circuit['0']

To extract the exact probabilities from a simulation we need to defer all measurements to the end. For this we add auxilliary qubits corresponding to each classical bit. Instead of measurements, we use `save_probabilities_dict()`.

We also need to rewrite the output bit string to reproduce the format that the result should be. The following functions do these things.

In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def move_msm(qc):
    
    bits = []
    for creg in qc.cregs:
        for bit in creg:
            bits.append(bit)

    new_qc = QuantumCircuit()

    for regs in [qc.qregs, qc.cregs]:
        for reg in regs:
            new_qc.add_register(reg)

    aux = {}
    for reg in qc.cregs:
        for bit in reg:
            aux[bits.index(bit)] = QuantumRegister(1)
            new_qc.add_register(aux[bits.index(bit)])

    for gate in qc.data:
        if gate[0].name=='measure':
            new_qc.cx(gate[1][0], aux[bits.index(gate[2][0])])
        else:
            new_qc.data.append(gate)
            
    new_qc.save_probabilities_dict()
        
    return new_qc, aux

def format_probs(probs, new_qc, aux):
    
    bits = []
    for creg in qc.cregs:
        for bit in creg:
            bits.append(bit)
    
    index = {}
    for reg in new_qc.cregs:
        for bit in reg:
            index[bit] = new_qc.qubits.index(aux[bits.index(bit)][0])

    new_probs = {}
    for string,prob in probs.items(): 
        new_string = ''
        for reg in new_qc.cregs:
            for bit in reg:
                j = index[bit]
                new_string += string[-1-j]
            new_string += ' '
        new_string = new_string[::-1][1::]
        if new_string in new_probs:
            new_probs[new_string] += prob
        else:
            new_probs[new_string] = prob
            
    return new_probs

Let's try it out.

In [4]:
backend = AerSimulator(zero_threshold=1e-5)

# make the modified circuit
new_qc, aux = move_msm(qc)

# run it
job = backend.run(new_qc)

# get the probabilities
probs = job.result().data(new_qc)['probabilities']

# since they use hex instead of output strings, convert to output strings
n = str(len(new_qc.qubits))
probs = {('{0:0'+n+'b}').format(int(output,0)):shots for output,shots in probs.items()}

# then use `format_probs`
probs = format_probs(probs, new_qc, aux)

# let's see what we get
probs

{'000 00 00': 1.0}